In [1]:
from dwave.cloud import Client


def read_token(file): #
    f = open(file, 'r')
    return f.read()

API_TOKEN = read_token('api_token')

client = Client.from_config(token=API_TOKEN)

In [2]:
from dwave.system import DWaveSampler

sample = DWaveSampler(solver={'qpu':True})


print(sample.parameters)

{'anneal_offsets': ['parameters'], 'anneal_schedule': ['parameters'], 'annealing_time': ['parameters'], 'answer_mode': ['parameters'], 'auto_scale': ['parameters'], 'beta': ['parameters'], 'chains': ['parameters'], 'flux_biases': ['parameters'], 'flux_drift_compensation': ['parameters'], 'h_gain_schedule': ['parameters'], 'initial_state': ['parameters'], 'max_answers': ['parameters'], 'num_reads': ['parameters'], 'num_spin_reversal_transforms': ['parameters'], 'postprocess': ['parameters'], 'programming_thermalization': ['parameters'], 'readout_thermalization': ['parameters'], 'reduce_intersample_correlation': ['parameters'], 'reinitialize_state': ['parameters'], 'warnings': [], 'label': []}


# Multi-Gate
https://docs.ocean.dwavesys.com/en/stable/examples/multi_gate.html

In [1]:
import dwavebinarycsp

def logic_circuit(a,b,c,d, z):
    # a,b,c,d are inputs
    # z is output
    not1 = not b
    or2 = b or c
    and3 = a and not1
    or4 = or2 or d
    and5 = and3 and or4
    not6 = not or4
    or7 = and5 or not6
    return (z == or7)

print("Defined Circuit")
# Constraint satisfaction problem
csp = dwavebinarycsp.ConstraintSatisfactionProblem(dwavebinarycsp.BINARY)

csp.add_constraint(logic_circuit, ['a', 'b', 'c', 'd', 'z'])
print("Defined CSP, computing bqm")


# Binary Quadratic Model
bqm = dwavebinarycsp.stitch(csp)
print("Finished stitching")

Defined Circuit
Defined CSP, computing bqm
Finished stitching


In [2]:
bqm

BinaryQuadraticModel({'a': 2.0, 'aux0': 0.0, 'aux1': 6.0, 'b': -2.0, 'c': 2.0, 'd': -2.0, 'z': -2.0}, {('aux0', 'a'): 0.0, ('aux1', 'a'): -4.0, ('aux1', 'aux0'): 2.0, ('b', 'a'): -2.0, ('b', 'aux0'): 4.0, ('b', 'aux1'): 4.0, ('c', 'a'): 0.0, ('c', 'aux0'): -4.0, ('c', 'aux1'): -2.0, ('c', 'b'): -2.0, ('d', 'a'): 0.0, ('d', 'aux0'): 2.0, ('d', 'aux1'): -2.0, ('d', 'b'): 2.0, ('d', 'c'): 0.0, ('z', 'a'): 0.0, ('z', 'aux0'): 4.0, ('z', 'aux1'): -4.0, ('z', 'b'): 4.0, ('z', 'c'): 0.0, ('z', 'd'): 4.0}, 2.0, 'BINARY')

In [3]:
from dwave.system import DWaveSampler, EmbeddingComposite

# Set up a D-Wave system as the sampler
sampler = EmbeddingComposite(DWaveSampler())


sampleset = sampler.sample(bqm, num_reads=1000)

# Check how many solutions meet the constraints (are valid)
print("Checking solutions")
valid, invalid, data = 0, 0, []
for datum in sampleset.data(['sample', 'energy', 'num_occurrences']):
    # print(datum.sample)
    if (csp.check(datum.sample)):
        valid = valid+datum.num_occurrences
        for i in range(datum.num_occurrences):
            data.append((datum.sample, datum.energy, '1'))
    else:
        invalid = invalid+datum.num_occurrences
        for i in range(datum.num_occurrences):
            data.append((datum.sample, datum.energy, '0'))

print(valid, invalid)        


Checking solutions
959 41


In [4]:
import dwave.inspector

dwave.inspector.show(sampleset)

ValueError: invalid combination of arguments provided: if data capture not enabled, problem/response/solver have to be specified; also, make sure a structured problem is being inspected

In [6]:
from dwave.system import DWaveSampler
import dwave.inspector

sampler = DWaveSampler(solver=dict(qpu=True))

h = {}
J = {(0, 4): 1, (0, 5): 1, (1, 4): 1, (1, 5): -1}

print(all(edge in sampler.edgelist for edge in J))
response = sampler.sample_ising(h, J, num_reads=100)

dwave.inspector.show(response)   


True
Opening in existing browser session.


'http://127.0.0.1:18001/?problemId=1bcf0175-5210-4aa2-91ec-5a34bceba979'